In [1]:
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large", forced_bos_token_id=0)
tok = BartTokenizer.from_pretrained("facebook/bart-large")
example_english_phrase = "A plane that is flying in the air during the day"
batch = tok(example_english_phrase, return_tensors="pt")
generated_ids = model.generate(batch["input_ids"])
print(tok.batch_decode(generated_ids, skip_special_tokens=True))

c:\Users\anand\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\anand\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
['A plane that is flying in the air during the day']


In [2]:
print(model.generation_config.force_words_ids)

None


In [3]:
output = model.forward(batch["input_ids"])

hi


In [4]:
decoded_output = model.model.decoder(inputs_embeds=output.encoder_last_hidden_state ,encoder_hidden_states=output.encoder_last_hidden_state)

In [5]:
decoded_output.keys()

odict_keys(['last_hidden_state', 'past_key_values'])

In [6]:
import torch

In [107]:
# Initialize the model and tokenizer
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

# Prepare the input
example_english_phrase = "picture of a white airplane flying in the sky is cool."
batch = tokenizer(example_english_phrase, return_tensors="pt")
input_ids = batch["input_ids"]

# Pass through the encoder
encoder_outputs = model.model.encoder(input_ids)
encoder_embeddings = encoder_outputs.last_hidden_state


In [108]:
decoder_input_ids = torch.full(
    (input_ids.shape[0], 1),
    model.config.decoder_start_token_id,
    dtype=torch.long,
    device=input_ids.device
)

In [109]:
encoder_outputs.last_hidden_state = torch.cat([encoder_outputs.last_hidden_state, torch.zeros((encoder_outputs.last_hidden_state.shape[0], 1,encoder_outputs.last_hidden_state.shape[2]))], dim=1)

In [110]:
output_ids = model.generate(encoder_outputs=encoder_outputs, decoder_input_ids=input_ids)

hi
hi
hi


In [111]:
print(tok.batch_decode(output_ids, skip_special_tokens=True))

['picture of a white airplane flying in the sky is cool.a']


In [ ]:

# Prepare initial decoder input (start with the decoder_start_token_id)
decoder_input_ids = torch.full(
    (input_ids.shape[0], 1),
    model.config.decoder_start_token_id,
    dtype=torch.long,
    device=input_ids.device
)

# Container for storing decoder embeddings
decoder_embeddings = []
max_decoder_steps = 50  # Limit to prevent infinite loops

# Run decoding in a loop
for step in range(max_decoder_steps):
    decoder_output = model.model.decoder(input_ids=decoder_input_ids,encoder_hidden_states=encoder_embeddings)

    # Get the embeddings of the last token produced
    last_token_embedding = decoder_output.last_hidden_state[:, -1, :]
    decoder_embeddings.append(last_token_embedding)

    # Predict the next token
    next_token_logits = model.lm_head(last_token_embedding)
    next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)
    print(next_token)

    # Check for end of sequence token
    if next_token.item() == tokenizer.eos_token_id:
        break

    # Append the predicted token to the decoder input
    decoder_input_ids = torch.cat([decoder_input_ids, next_token], dim=-1)

# Convert the list of tensors to a tensor
decoder_embeddings = torch.stack(decoder_embeddings, dim=1)

# Decode the generated token IDs back into text
decoded_text = tokenizer.decode(decoder_input_ids.squeeze(), skip_special_tokens=True)

print("Decoded Text:", decoded_text)
print("Decoder Embeddings Shape:", decoder_embeddings.shape)